#Initialize

In [ ]:
!pip install pytorch-lightning==1.6.5
!pip install pandas numpy opencv-python Pillow scikit-image scikit-learn scipy tqdm fairseq wandb statsmodels
%cd /content/drive/Shareddrives/THESIS/TEST_CODE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 44.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.4/240.4 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import argparse
import torch
import pandas as pd
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from pytorch_lightning import loggers as pl_loggers
from datetime import datetime

from dataset import Visuelle2
from models.CrossAttnRNN21 import CrossAttnRNN as Model21
from models.CrossAttnRNN210 import CrossAttnRNN as Model210
from models.CrossAttnRNNDemand import CrossAttnRNN as DemandModel

In [ ]:
def run(args):
    print(args)

    # Seed for reproducibility (By default we use the number 21)
    pl.seed_everything(args.seed)

    ####################################### Load data #######################################
    # Load train-test data
    train_df = pd.read_csv(
        os.path.join(args.dataset_path, "stfore_train.csv"),
        parse_dates=["release_date"],
    )

    test_df = pd.read_csv(
        os.path.join(args.dataset_path, "stfore_test.csv"),
        parse_dates=["release_date"],
    )

    # Load attribute encodings
    cat_dict = torch.load(os.path.join(args.dataset_path, "category_labels.pt"))
    col_dict = torch.load(os.path.join(args.dataset_path, "color_labels.pt"))
    fab_dict = torch.load(os.path.join(args.dataset_path, "fabric_labels.pt"))

    # Load Google trends
    gtrends = pd.read_csv(
        os.path.join(args.dataset_path, "vis2_gtrends_data.csv"), index_col=[0], parse_dates=True
    )

    demand = bool(args.demand)
    img_folder = os.path.join(args.dataset_path, 'images')

    if args.new_window == True:
        if demand:
            visuelle_pt_train = "visuelle2_train_processed_demand_4.pt"  
            visuelle_pt_test = "visuelle2_test_processed_demand_4.pt"  
        else:
            visuelle_pt_train = "visuelle2_train_processed_stfore_4.pt"
            visuelle_pt_test = "visuelle2_test_processed_stfore_4.pt"
    else:
        if demand:
            visuelle_pt_train = "visuelle2_train_processed_demand_2.pt"  
            visuelle_pt_test = "visuelle2_test_processed_demand_2.pt"  
        else:
            visuelle_pt_train = "visuelle2_train_processed_stfore_2.pt"
            visuelle_pt_test = "visuelle2_test_processed_stfore_2.pt" 

    # Create (PyTorch) dataset objects
    trainset = Visuelle2(
        sales_df=train_df,
        img_root=img_folder,
        gtrends=gtrends,
        cat_dict=cat_dict,
        col_dict=col_dict,
        fab_dict=fab_dict,       
        trend_len=52,
        new_window=args.new_window,        
        demand=demand,
        local_savepath=os.path.join(args.dataset_path, visuelle_pt_train)
    )
    testset = Visuelle2(
        sales_df=test_df,
        img_root=img_folder,
        gtrends=gtrends,
        cat_dict=cat_dict,
        col_dict=col_dict,
        fab_dict=fab_dict,
        trend_len=52,   
        new_window=args.new_window,        
        demand=demand,
        local_savepath=os.path.join(args.dataset_path, visuelle_pt_test)
    )

    # If you wish to debug with less data you can use this
    if args.quick_debug:
        trainset = torch.utils.data.Subset(trainset, list(range(1000)))
        testset = torch.utils.data.Subset(testset, list(range(1000)))

    trainloader = DataLoader(
        trainset, batch_size=args.batch_size, shuffle=True, num_workers=6
    )

    testloader = DataLoader(
        testset, batch_size=args.batch_size, shuffle=False, num_workers=4
    )

    print(f"Completed dataset loading procedure. Train batches: {len(trainloader)}, test batches: {len(testloader)}")

    # ####################################### Train and eval model #######################################
    model = None
    if demand:
        model = DemandModel(
            attention_dim=args.attention_dim,
            embedding_dim=args.embedding_dim,
            hidden_dim=args.hidden_dim,
            num_trends=args.num_trends,
            cat_dict=cat_dict,
            col_dict=col_dict,
            fab_dict=fab_dict,
            store_num=125,
            use_img=args.use_img,
            use_att=args.use_att, 
            use_date=args.use_date,
            use_trends=args.use_trends,
            use_teacher_forcing=args.use_teacher_forcing,
            teacher_forcing_ratio=args.teacher_forcing_ratio,
            out_len=12, # Demand predicts the full series in one go
        )
    else:
        if args.task_mode == 0:
            model = Model21(
                attention_dim=args.attention_dim,
                embedding_dim=args.embedding_dim,
                hidden_dim=args.hidden_dim,
                use_img=args.use_img,
                out_len=args.output_len,
            ) 
        else:
            model = Model210(
                attention_dim=args.attention_dim,
                embedding_dim=args.embedding_dim,
                hidden_dim=args.hidden_dim,
                use_img=args.use_img,
                out_len=args.output_len,
                use_teacher_forcing=args.use_teacher_forcing,
                teacher_forcing_ratio=args.teacher_forcing_ratio,
            )       

    # Define model saving procedure
    dt_string = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
    model_savename = args.wandb_run
    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        dirpath=args.ckpt_dir,
        filename=model_savename + "---{epoch}---" + dt_string,
        monitor="val_wWAPE",
        mode="min",
        save_top_k=2,
    )

    if args.use_wandb:
        wandb_logger = pl_loggers.WandbLogger(
            project=args.wandb_project, entity=args.wandb_entity, name=args.wandb_run
        )

    trainer = pl.Trainer(
        gpus=[args.gpu_num],
        max_epochs=args.epochs,
        check_val_every_n_epoch=1,
        logger=wandb_logger if args.use_wandb else None,
        callbacks=[checkpoint_callback]
    )

    # Fit model
    trainer.fit(model, train_dataloaders=trainloader, val_dataloaders=testloader)

    # Print out path of best model
    print(checkpoint_callback.best_model_path)


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset_path", type=str, default='visuelle2/')
parser.add_argument("--seed", type=int, default=21)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--demand", type=int, default=0,
help="Boolean variable to optionally use the dataset for the new product demand forecasting task (forecasting without a known past)")
parser.add_argument("--quick_debug", action='store_true')

# Model specific arguments
parser.add_argument("--embedding_dim", type=int, default=512)
parser.add_argument("--attention_dim", type=int, default=512)
parser.add_argument("--hidden_dim", type=int, default=512)
parser.add_argument("--output_len", type=int, default=10)
parser.add_argument("--use_img", type=bool, default=True)
parser.add_argument("--use_att", type=bool, default=True)
parser.add_argument("--use_date", type=bool, default=True)
parser.add_argument("--use_trends", type=bool, default=True)
parser.add_argument("--num_trends", type=int, default=3)
parser.add_argument("--task_mode", type=int, default=0, help="0-->2-1 - 1-->2-10")
parser.add_argument("--new_window", type=str, default=False)
parser.add_argument("--epochs", type=int, default=30)
parser.add_argument("--gpu_num", type=int, default=0)
parser.add_argument("--use_teacher_forcing", action='store_true')
parser.add_argument("--teacher_forcing_ratio", type=float, default=0.5)

# wandb arguments
parser.add_argument("--use_wandb", action='store_true')
parser.add_argument("--wandb_entity", type=str, default="")
parser.add_argument("--wandb_project", type=str, default="")
parser.add_argument("--wandb_run", type=str, default="")
parser.add_argument("--ckpt_dir", type=str, default="ckpt/")

parser.add_argument('-f')
args = parser.parse_args()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# SO-fore 4-1 RNN w/image attention

In [ ]:
# epochs = 5
# Run SO-fore4−1 train_window = 4
# Output: create ckpt-path
if __name__ == "__main__":
  args.epochs = 5
  args.new_window = True
  run(args)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


Namespace(attention_dim=512, batch_size=128, ckpt_dir='ckpt/', dataset_path='visuelle2/', demand=0, embedding_dim=512, epochs=5, f='/root/.local/share/jupyter/runtime/kernel-95ed3a24-ad93-4706-b72d-1b1d0bcedeb3.json', gpu_num=0, hidden_dim=512, new_window=True, num_trends=3, output_len=10, quick_debug=False, seed=21, task_mode=0, teacher_forcing_ratio=0.5, use_att=True, use_date=True, use_img=True, use_teacher_forcing=False, use_trends=True, use_wandb=False, wandb_entity='', wandb_project='', wandb_run='')
Loading dataset...
Done.
Loading dataset...
Done.
Completed dataset loading procedure. Train batches: 752, test batches: 84


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can al

  0%|          | 0.00/104M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/drive/Shareddrives/THESIS/TEST_CODE/ckpt exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type              | Params
----------------------------------------------------
0 | image_encoder | ImageEncoder      | 26.2 M
1 | ts_embedder   | GRU               | 791 K 
2 | img_attention | AdditiveAttention | 524 K 
3 | decoder       | Sequential    

...

Validation MAE: 0.9796015 Validation WAPE: 94.14255 LR: tensor(5.9311e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.0562854 Validation WAPE: 101.5121 LR: tensor(8.1763e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 0.9558152 Validation WAPE: 91.85662 LR: tensor(0.0001, device='cuda:0')
/content/drive/Shareddrives/THESIS/TEST_CODE/ckpt/---epoch=4---04-02-2023-18-54-08.ckpt


# SO-fore 2-1 RNN w/image attention

In [ ]:
# epochs = 5
# Run SO-fore2−1 train_window = 2
# Output: create ckpt-path
if __name__ == "__main__":
  args.epochs = 5
  args.new_window = False
  run(args)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


Namespace(attention_dim=512, batch_size=128, ckpt_dir='ckpt/', dataset_path='visuelle2/', demand=0, embedding_dim=512, epochs=5, f='/root/.local/share/jupyter/runtime/kernel-95ed3a24-ad93-4706-b72d-1b1d0bcedeb3.json', gpu_num=0, hidden_dim=512, new_window=False, num_trends=3, output_len=10, quick_debug=False, seed=21, task_mode=0, teacher_forcing_ratio=0.5, use_att=True, use_date=True, use_img=True, use_teacher_forcing=False, use_trends=True, use_wandb=False, wandb_entity='', wandb_project='', wandb_run='')
Loading dataset...
Done.
Loading dataset...
Done.
Completed dataset loading procedure. Train batches: 752, test batches: 84


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type              | Params
----------------------------------------------------
0 | image_encoder | ImageEncoder      | 26.2 M
1 | ts_embedder   | GRU               | 791 K 
2 | img_attention | AdditiveAttention | 524 K 
3 | decoder       | Sequential        | 394 K 
4 | decoder_gru   | GRU               | 4.7 M 
----------------------------------------------------
18.6 M    Trainable params
14.0 M    Non-trainable params
32.6 M    Total params
130.397   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Validation MAE: 2.1784518 Validation WAPE: 123.68233 LR: None


...

Validation MAE: 0.9997979 Validation WAPE: 90.08966 LR: tensor(5.8899e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.0164187 Validation WAPE: 91.58732 LR: tensor(8.3302e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 0.97789216 Validation WAPE: 88.11578 LR: tensor(0.0001, device='cuda:0')
/content/drive/Shareddrives/THESIS/TEST_CODE/ckpt/---epoch=4---04-02-2023-20-29-53.ckpt


# SO-fore 4-8 w/image attention

In [ ]:
# Run SO-fore4−8 train_window = 4
# Output: create ckpt-path
if __name__ == "__main__":
  args.epochs = 5
  args.task_mode = 1
  args.new_window = True
  args.output_len = 8
  run(args)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


Namespace(attention_dim=512, batch_size=128, ckpt_dir='ckpt/', dataset_path='visuelle2/', demand=0, embedding_dim=512, epochs=5, f='/root/.local/share/jupyter/runtime/kernel-5d07b4c3-d971-45f4-aa26-7bab6fca201f.json', gpu_num=0, hidden_dim=512, new_window=True, num_trends=3, output_len=8, quick_debug=False, seed=21, task_mode=1, teacher_forcing_ratio=0.5, use_att=True, use_date=True, use_img=True, use_teacher_forcing=False, use_trends=True, use_wandb=False, wandb_entity='', wandb_project='', wandb_run='')
Loading dataset...
Done.
Loading dataset...
Done.
Completed dataset loading procedure. Train batches: 752, test batches: 84


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                  | Type              | Params
------------------------------------------------------------
0 | image_encoder         | ImageEncoder      | 26.2 M
1 | ts_embedder           | GRU               | 791 K 
2 | img_attention         | AdditiveAttention | 524 K 
3 | attentive_aggregator1 | Linear            | 263 K 
4 | attentive_aggregator2 | Linear            | 525 K 
5 | decoder_gru           | GRU               | 4.7 M 
6 | decoder_fc            | Sequential        | 131 K 
----------------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Validation MAE: 1.8910749 Validation WAPE: 106.25298 LR: None


...

Validation MAE: 1.084935 Validation WAPE: 104.265396 LR: tensor(5.8325e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 0.955778 Validation WAPE: 91.85304 LR: tensor(7.7317e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 0.95097333 Validation WAPE: 91.391304 LR: tensor(9.3875e-05, device='cuda:0')
/content/drive/Shareddrives/THESIS/MAIN/visuelle2-code/ckpt/---epoch=4---30-01-2023-14-25-19.ckpt


# SO-fore 2-10 w/image attention

In [ ]:
# Run SO-fore2−10 train_window = 2
# Output: create ckpt-path
if __name__ == "__main__":
  args.epochs = 5
  args.new_window = False
  args.task_mode = 1
  args.output_len = 10
  run(args)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


Namespace(attention_dim=512, batch_size=128, ckpt_dir='ckpt/', dataset_path='visuelle2/', demand=0, embedding_dim=512, epochs=5, f='/root/.local/share/jupyter/runtime/kernel-5d07b4c3-d971-45f4-aa26-7bab6fca201f.json', gpu_num=0, hidden_dim=512, new_window=False, num_trends=3, output_len=10, quick_debug=False, seed=21, task_mode=1, teacher_forcing_ratio=0.5, use_att=True, use_date=True, use_img=True, use_teacher_forcing=False, use_trends=True, use_wandb=False, wandb_entity='', wandb_project='', wandb_run='')
Loading dataset...
Done.
Loading dataset...
Done.
Completed dataset loading procedure. Train batches: 752, test batches: 84


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                  | Type              | Params
------------------------------------------------------------
0 | image_encoder         | ImageEncoder      | 26.2 M
1 | ts_embedder           | GRU               | 791 K 
2 | img_attention         | AdditiveAttention | 524 K 
3 | attentive_aggregator1 | Linear            | 263 K 
4 | attentive_aggregator2 | Linear            | 525 K 
5 | decoder_gru           | GRU               | 4.7 M 
6 | decoder_fc            | Sequential        | 131 K 
----------------------------------------------------

Sanity Checking: 0it [00:00, ?it/s]

Validation MAE: 1.8644917 Validation WAPE: 105.85715 LR: None


...

Validation MAE: 1.0166503 Validation WAPE: 91.60819 LR: tensor(5.8357e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.0741096 Validation WAPE: 96.78573 LR: tensor(7.8568e-05, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.0544583 Validation WAPE: 95.01499 LR: tensor(9.9105e-05, device='cuda:0')
/content/drive/Shareddrives/THESIS/MAIN/visuelle2-code/ckpt/---epoch=1---30-01-2023-15-59-08.ckpt


# Demand SO

In [ ]:
# Run Demand SO
# Output: create ckpt-path
if __name__ == "__main__":
  args.batch_size = 64
  args.epochs = 5
  args.demand = 1
  args.new_window = False
  run(args)

INFO:pytorch_lightning.utilities.seed:Global seed set to 21


Namespace(attention_dim=512, batch_size=64, ckpt_dir='ckpt/', dataset_path='visuelle2/', demand=1, embedding_dim=512, epochs=5, f='/root/.local/share/jupyter/runtime/kernel-5d07b4c3-d971-45f4-aa26-7bab6fca201f.json', gpu_num=0, hidden_dim=512, new_window=False, num_trends=3, output_len=10, quick_debug=False, seed=21, task_mode=1, teacher_forcing_ratio=0.5, use_att=True, use_date=True, use_img=True, use_teacher_forcing=False, use_trends=True, use_wandb=False, wandb_entity='', wandb_project='', wandb_run='')
Loading dataset...
Done.
Loading dataset...
Done.
Completed dataset loading procedure. Train batches: 1503, test batches: 167


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                 | Type                   | Params
-----------------------------------------------------------------
0  | trend_encoder        | TSEmbedder             | 794 K 
1  | temp_encoder         | TemporalFeatureEncoder | 4.1 K 
2  | image_encoder        | ImageEncoder           | 43.5 M
3  | attribute_encoder    | AttributeEncoder       | 117 K 
4  | ts_self_attention    | MultiheadAttention     | 1.1 M 
5  | ts_attention         | AdditiveAttention      | 524 K 
6  | trend_linear         | Linear                 | 13.6 M
7  | im

...

Validation MAE: 0.9931155 Validation WAPE: 84.267845 LR: tensor(0.0002, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.8381892 Validation WAPE: 155.97406 LR: tensor(0.0002, device='cuda:0')


Validation: 0it [00:00, ?it/s]

Validation MAE: 1.3957671 Validation WAPE: 118.43365 LR: tensor(0.0003, device='cuda:0')
/content/drive/Shareddrives/THESIS/MAIN/visuelle2-code/ckpt/---epoch=2---30-01-2023-17-21-36.ckpt
